# NLP Intent Recognition

Hallo und herzlich willkommen zum codecentric.AI bootcamp!

Heute wollen wir uns mit einem fortgeschrittenen Thema aus dem Bereich _natural language processing_, kurz _NLP_, genannt, beschäftigen:

> Wie bringt man Sprachassistenten, Chatbots und ähnlichen Systemen bei, die Absicht eines Nutzers aus seinen Äußerungen zu erkennen?

Dieses Problem wird im Englischen allgemein als _intent recognition_ bezeichnet und gehört zu dem ambitionierten Gebiet des _natural language understanding_, kurz _NLU_ genannt. Einen Einstieg in dieses Thema bietet das folgende [Youtube-Video](https://www.youtube.com/watch?v=H_3R8inCOvM):

In [ ]:
# lade Video
from IPython.display import IFrame    
IFrame('https://www.youtube.com/embed/H_3R8inCOvM', width=850, height=650)

Zusammen werden wir in diesem Tutorial mit Hilfe der NLU-Bibliothek [Rasa-NLU](https://rasa.com/docs/nlu/) einem WetterBot beibringen, einfache Fragemuster zum Wetter zu verstehen und zu beantworten. Zum Beispiel wird er auf die Fragen

> `"Wie warm war es 1989?"`

mit

> <img src="img/answer-1.svg" width="75%" align="middle">

und auf

> `"Welche Temperatur hatten wir in Schleswig-Holstein und in Baden-Württemberg?"`

mit

>  <img src="img/answer-2.svg" width="75%" align="middle">

antworten.

Damit es gleich richtig losgehen kann, importieren wir noch zwei Standardbibliotheken und vereinbaren das Datenverzeichnis:

In [ ]:
import os
import numpy as np


DATA_DIR = 'data'

## Unser Ausgangspunkt

Allgemein ist die Aufgabe, aus einer Sprachäußerung die zugrunde liegende Absicht zu erkennen, selbst für Menschen manchmal nicht einfach. Soll ein Computer diese schwierige Aufgabe lösen, so muss man sich überlegen, was man zu einem gegebenen Input &mdash; also einer (unstrukturierten) Sprachäußerung &mdash; für einen Output erwarten, wie man also Absichten modelliert und strukturiert.

Weit verbreitet ist folgender Ansatz für Intent Recognition:

- jede Äußerung wird einer _Domain_, also einem Gebiet, zugeordnet,
- für jede _Domain_ gibt es einen festen Satz von _Intents_, also eine Reihe von Absichten,
- jede Absicht kann durch _Parameter_ konkretisiert werden und hat dafür eine Reihe von _Slots_, die wie Parameter einer Funktion oder Felder eines Formulares mit gewissen Werten gefüllt werden können.

Für die Äußerungen

>  - `"Wie warm war es 1990 in Berlin?"`
>  - `"Welche Temperatur hatten wir in Hessen im Jahr 2018?"`
>  - `"Wie komme ich zum Hauptbahnhof?"`

könnte _Intent Recognition_ also zum Beispiel jeweils folgende Ergebnisse liefern:

> - `{'intent': 'Frag_Temperatur', 'slots': {'Ort': 'Berlin', 'Jahr': '1990'}}`
> - `{'intent': 'Frag_Temperatur', 'slots': {'Ort': 'Hessen', 'Jahr': '2018'}}`
> - `{'intent': 'Frag_Weg', 'slots': {'Start': None, 'Ziel': 'Hauptbahnhof'}}`

Für Python steht eine ganze von NLP-Bibliotheken zur Verfügung, die Intent Recognition in der einen oder anderen Form ermöglichen, zum Beispiel

- [Rasa NLU](https://rasa.com/docs/nlu/) (&bdquo;Language Understanding for chatbots and AI assistants&ldquo;),
- [snips](https://snips-nlu.readthedocs.io/en/latest/) (&bdquo;Using Voice to Make Technology Disappear&ldquo;),
- [DeepPavlov](http://deeppavlov.ai) (&bdquo;an open-source conversational AI library&ldquo;),
- [NLP Architect](http://nlp_architect.nervanasys.com/index.html) von Intel (&bdquo;for exploring state-of-the-art deep learning topologies and techniques for natural language processing and natural language unterstanding&ldquo;),
- [pytext](https://pytext-pytext.readthedocs-hosted.com/en/latest/index.html) von Facebook (&bdquo;a deep-learning based NLP modeling framework built on PyTorch&ldquo;).

Wir entscheiden uns im Folgenden für die Bibliothek Rasa NLU, weil wir dafür bequem mit einem Open-Source-Tool (chatette) umfangreiche Trainingsdaten generieren können. Rasa NLU wiederum benutzt  die NLP-Bibliothek [spaCy](https://spacy.io), die Machine-Learning-Bibliothek [scikit-learn](https://scikit-learn.org/stable/) und die Deep-Learning-Bibliothek [TensorFlow](https://www.tensorflow.org/).


## Intent Recognition von Anfang bis Ende mit Rasa NLU

Schauen wir uns an, wie man eine Sprach-Engine für Intent Recognition trainieren kann! Dafür beschränken wir uns zunächst auf wenige Intents und Trainingsdaten und gehen die benötigten Schritte von Anfang bis Ende durch.

### Schritt 1: Intents durch Trainingsdaten beschreiben

Als Erstes müssen wir die Intents mit Hilfe von Trainingsdaten beschreiben. _Rasa NLU_ erwartet beides zusammen in einer Datei im menschenfreundlichen [Markdown-Format](http://markdown.de/) oder im computerfreundlichen [JSON-Format](https://de.wikipedia.org/wiki/JavaScript_Object_Notation). Ein Beispiel für solche Trainingsdaten im Markdown-Format ist der folgende Python-String, den wir in die Datei `intents.md` speichern: 

In [ ]:
TRAIN_INTENTS = """
## intent: Frag_Temperatur
- Wie [warm](Eigenschaft) war es [1900](Zeit) in [Brandenburg](Ort)
- Wie [kalt](Eigenschaft) war es in [Hessen](Ort) [1900](Zeit)
- Was war die Temperatur [1977](Zeit) in [Sachsen](Ort)

## intent: Frag_Ort
- Wo war es [1998](Zeit) am [kältesten](Superlativ:kalt)
- Finde das [kältesten](Superlativ:kalt) Bundesland im Jahr [2004](Zeit)
- Wo war es [2010](Zeit) [kälter](Komparativ:kalt) als [1994](Zeit) in [Rheinland-Pfalz](Ort)

## intent: Frag_Zeit
- Wann war es in [Bayern](Ort) am [kühlsten](Superlativ:kalt)
- Finde das [kälteste](Superlativ:kalt) Jahr im [Saarland](Ort)
- Wann war es in [Schleswig-Holstein](Ort) [wärmer](Komparativ:warm) als in [Baden-Württemberg](Ort)

## intent: Ende
- Ende
- Auf Wiedersehen
- Tschuess
"""


INTENTS_PATH = os.path.join(DATA_DIR, 'intents.md')


def write_file(filename, text):
    with open(filename, 'w', encoding='utf-8') as file:
        file.write(text)

write_file(INTENTS_PATH, TRAIN_INTENTS)

Hier wird jeder Intent erst in der Form

> `## intent: NAME`

deklariert, wobei `NAME` durch die Bezeichnung des Intents zu ersetzen ist. Anschließend wird der Intent durch eine Liste von
Beispiel-Äußerungen beschrieben. Die Parameter beziehungsweise Slots werden in den Beispieläußerungen in der Form

> `[WERT](SLOT)`

markiert, wobei `SLOT` die Bezeichnung des Slots und `Wert` der entsprechende Teil der Äußerung ist.


### Schritt 2: Sprach-Engine konfigurieren...

Die Sprach-Engine von _Rasa NLU_ ist als Pipeline gestaltet und [sehr flexibel konfigurierbar](https://rasa.com/docs/nlu/components/#section-pipeline). Zwei [Beispiel-Konfigurationen](https://rasa.com/docs/nlu/choosing_pipeline/) sind in Rasa bereits enthalten:

- `spacy_sklearn` verwendet vortrainierte Wortvektoren, eine [scikit-learn-Implementierung](https://scikit-learn.org/stable/modules/svm.html) einer linearen [Support-vector Machine]( https://en.wikipedia.org/wiki/Support-vector_machine) für die Klassifikation und wird für kleine Trainingsmengen (<1000) empfohlen. Da diese Pipeline vortrainierte Wortvektoren und spaCy benötigt, kann sie nur für [die meisten westeuropäische Sprachen](https://rasa.com/docs/nlu/languages/#section-languages) verwendet werden.

- `tensorflow_embedding` trainiert für die Klassifikation Einbettungen von Äußerungen und von Intents in denselben Vektorraum  und wird für größere Trainingsmengen (>1000) empfohlen. Die zu Grunde liegende Idee stammt aus dem Artikel [StarSpace: Embed All The Things!](https://arxiv.org/abs/1709.03856). Sie ist sehr vielseitig anwendbar und beispielsweise auch für  [Question Answering](https://en.wikipedia.org/wiki/Question_answering) geeignet. Diese Pipeline benötigt kein Vorwissen über die verwendete Sprache, ist also universell einsetzbar, und kann auch auf das Erkennen mehrerer Intents in einer Äußerung trainiert werden.

Zum Füllen der Slots verwenden beide Pipelines eine [Python-Implementierung](http://www.chokkan.org/software/crfsuite/) von [Conditional Random Fields](https://en.wikipedia.org/wiki/Conditional_random_field).

Die Konfiguration der Pipeline wird durch eine YAML-Datei beschrieben. Der folgende Python-String entspricht der Variante `spacy_sklearn`:

In [ ]:
CONFIG_SK = """
language: de_core_news_sm
pipeline:
- name: "nlp_spacy"
  case_sensitive: true
- name: "tokenizer_spacy"
- name: "intent_entity_featurizer_regex"
- name: "intent_featurizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_classifier_sklearn"
"""


### Schritt 3: ...trainieren...

Sind die Trainingsdaten und die Konfiguration der Pipeline beisammen, so kann die Sprach-Engine trainiert werden. In der Regel erfolgt dies bei Rasa mit Hilfe eines Kommandozeilen-Interface oder direkt [in Python](https://rasa.com/docs/nlu/python/). Die folgende Funktion `train` erwartet die Konfiguration als Python-String und den Namen der Datei mit den Trainingsdaten und gibt die trainierte Sprach-Engine als Instanz einer `Interpreter`-Klasse zurück:

In [ ]:
import rasa_nlu.training_data
import rasa_nlu.config
from rasa_nlu.model import Trainer, Interpreter

MODEL_DIR = 'models'

def train(config=CONFIG_SK, intents_path=INTENTS_PATH):
    config_path = os.path.join(DATA_DIR, 'rasa_config.yml')
    write_file(config_path, config)
    trainer = Trainer(rasa_nlu.config.load(config_path))
    trainer.train(rasa_nlu.training_data.load_data(intents_path))
    return Interpreter.load(trainer.persist(MODEL_DIR))

interpreter = train()

### Schritt 4: ...und testen!

Wir testen nun, ob die Sprach-Engine `interpreter` folgende Test-Äußerungen richtig versteht:

In [ ]:
TEST_UTTERANCES = [
    'Was war die durchschnittliche Temperatur 2004 in Mecklenburg-Vorpommern',
    'Nenn mir das wärmste Bundesland 2018',
    'In welchem Jahr war es in Nordrhein-Westfalen heißer als 1990',
    'Wo war es 2000 am kältesten',
    'Bis bald',
]

Die Methode `parse` von `interpreter` erwartet eine Äußerung als Python-String, wendet Intent Recognition an und liefert eine sehr detaillierte Rückgabe:

In [ ]:
interpreter.parse(TEST_UTTERANCES[0])

Die Rückgabe umfasst im Wesentlichen

- den Namen des ermittelten Intent sowie eine Sicherheit beziehungsweise Konfidenz zwischen 0 und 1,
- für jeden ermittelten Parameter die Start- und Endposition in der Äußerung, den Wert und wieder eine Konfidenz,
- ein Ranking der möglichen Intents nach der Sicherheit/Konfidenz, mit der sie in dieser Äußerung vermutet wurden.

Für eine übersichtlichere Darstellung und leichte Weiterverarbeitung bereiten wir die Rückgabe mit Hilfe der Funktionen `extract_intent` und `extract_confidences` ein wenig auf. Anschließend gehen wir unsere Test-Äußerungen durch:

In [ ]:
def extract_intent(intent):
    return (intent['intent']['name'] if intent['intent'] else None,
            [(ent['entity'], ent['value']) for ent in intent['entities']])


def extract_confidences(intent):
    return (intent['intent']['confidence'] if intent['intent'] else None,
           [ent['confidence'] for ent in intent['entities']])


def test(interpreter, utterances=TEST_UTTERANCES):
    for utterance in utterances:
        intent = interpreter.parse(utterance)
        print('<', utterance)
        print('>', extract_intent(intent))
        print(' ', extract_confidences(intent))
        print()

test(interpreter)

Das Ergebnis ist noch nicht ganz überzeugend &mdash;  wir haben aber auch nur ganz wenig Trainingsdaten vorgegeben!

##  Trainingsdaten generieren mit Chatette

Für ein erfolgreiches Training brauchen wir also viel mehr Trainingsdaten. Doch fängt man an, weitere Beispiele aufzuschreiben, so fallen einem schnell viele kleine Variationsmöglichkeiten ein, die sich recht frei kombinieren lassen. Zum Beispiel können wir für eine Frage nach der Temperatur in Berlin im Jahr 1990 mit jeder der Phrasen
> - "Wie warm war es..."
> - "Wie kalt war es..."
> - "Welche Temperatur hatten wir..."

beginnen und dann mit

> - "...in Berlin 1990"
> - "...1990 in Berlin"

abschließen, vor "1990" noch "im Jahr" einfügen und so weiter. Statt alle denkbaren Kombinationen aufzuschreiben, ist es sinnvoller, die Möglichkeiten mit Hilfe von Regeln zu beschreiben und daraus Trainingsdaten generieren zu lassen. Genau das ermöglicht das Python-Tool [chatette](https://github.com/SimGus/Chatette), das wir im Folgenden verwenden. Dieses Tool liest Regeln, die einer speziellen Syntax folgen müssen, aus einer Datei aus und erzeugt dann daraus Trainingsdaten für Rasa NLU im JSON-Format.


### Regeln zur Erzeugung von Trainingsdaten

Wir legen im Folgenden erst einen Grundvorrat an Regeln für die Intents `Frag_Temperatur`, `Frag_Ort`, `Frag_Zeit` und `Ende` in einem Python-Dictionary an und erläutern danach genauer, wie die Regeln aufgebaut sind:

In [ ]:
RULES = {
    '@[Ort]': (
        'Brandenburg', 'Baden-Wuerttemberg', 'Bayern', 'Hessen',
        'Rheinland-Pfalz', 'Schleswig-Holstein', 'Saarland', 'Sachsen',
    ),
    '@[Zeit]': set(map(str, np.random.randint(1891, 2018, size=5))),
    '@[Komparativ]': ('wärmer', 'kälter',),
    '@[Superlativ]': ('wärmsten', 'kältesten',),
    '%[Frag_Temperatur]': ('Wie {warm|kalt} war es ~[zeit_ort]',
                  'Welche Temperatur hatten wir ~[zeit_ort]',
                  'Wie war die Temperatur ~[zeit_ort]',
    ),
    '%[Frag_Ort]': (
        '~[wo_war] es @[Zeit] @[Komparativ] als {@[Zeit]|in @[Ort]}',
        '~[wo_war] es @[Zeit] am @[Superlativ]',
    ),
    '%[Frag_Jahr]': (
        '~[wann_war] es in @[Ort] @[Komparativ] als {@[Zeit]|in @[Ort]}',
        '~[wann_war] es in @[Ort] am @[Superlativ]',
    ),
    '%[Ende]': ('Ende', 'Auf Wiedersehen', 'Tschuess',),
    '~[finde]': ('Sag mir', 'Finde'),
    '~[wie_war]': ('Wie war', '~[finde]',),
    '~[was_war]': ('Was war', '~[finde]',),
    '~[wo_war]': ('Wo war', 'In welchem {Bundesland|Land} war',),
    '~[wann_war]': ('Wann war', 'In welchem Jahr war',),
    '~[zeit_ort]': ('@[Zeit] in @[Ort]', '@[Ort] in @[Zeit]',),
    '~[Bundesland]': ('Land', 'Bundesland',),
}

Jede Regel besteht aus einem Namen beziehungsweise Platzhalter und einer Menge von Phrasen. Je nachdem, ob der Name die Form
> `%[NAME]`, `@[NAME]` oder `~[NAME]`

hat, beschreibt die Regel einen

> _Intent_, _Slot_ oder eine _Alternative_

mit der Bezeichnung `NAME`.  Jede Phrase kann ihrerseits Platzhalter für Slots und Alternativen erhalten. Diese Platzhalter werden bei der Erzeugung von Trainingsdaten von chatette jeweils durch eine der Phrasen ersetzt, die in der Regel für den jeweiligen Slot beziehungsweise die Alternativen aufgelistet sind. Außerdem können Phrasen

- Alternativen der Form `{_|_|_}`,
- optionale Teile in der Form `[_?]`

und einige weitere spezielle Konstrukte enthalten. Mehr Details finden sich in der [Syntax-Beschreibung](https://github.com/SimGus/Chatette/wiki/Syntax-specifications) von chatette.



### Erzeugung der Trainingsdaten

Die in dem Python-Dictionary kompakt abgelegten Regeln müssen nun für chatette so formatiert werden, dass bei jeder Regel der Name einen neuen Absatz einleitet und anschließend die möglichen Phrasen schön eingerückt Zeile für Zeile aufgelistet werden. Dies leistet die folgende Funktion `format_rules`.  Zusätzlich fügt sie eine Vorgabe ein, wieviel Trainingsbeispiele pro Intent erzeugt werden sollen:

In [ ]:
def format_rules(rules, train_samples):
    train_str =  "('training':'{}')".format(train_samples)
    llines = [[name if (name[0] != '%') else name + train_str]
              + ['    ' + val for val in rules[name]] + [''] for name in rules]
    return '\n'.join((l for lines in llines for l in lines))


Nun wenden wir chatette an, um die Trainingsdaten zu generieren. Dafür bietet chatette ein bequemes [Kommandozeilen-Interface](https://github.com/SimGus/Chatette/wiki/Command-line-interface), aber wir verwenden direkt die zu Grunde liegenden Python-Module.

Die folgende Funktion `chatette` erwartet wie `format_rules` ein Python-Dictionary mit Regeln, schreibt diese passend formatiert in eine Datei, löscht etwaige zuvor generierte Trainingsdateien und erzeugt dann den Regeln entsprechend neue Trainingsdaten.

In [ ]:
from chatette.adapters import RasaAdapter
from chatette.parsing import Parser
from chatette.generator import Generator
import glob

TRAIN_SAMPLES = 400
CHATETTE_DIR = os.path.join(DATA_DIR, 'chatette')


def chatette(rules=RULES, train_samples=TRAIN_SAMPLES):
    rules_path = os.path.join(DATA_DIR, 'intents.chatette')
    write_file(rules_path, format_rules(rules, train_samples))
    with open(rules_path, 'r') as rule_file:
        parser = Parser(rule_file)
        parser.parse()
    generator = Generator(parser)
    for f in glob.glob(os.path.join(CHATETTE_DIR, '*')):
        os.remove(f)
    RasaAdapter().write(CHATETTE_DIR, list(generator.generate_train()),
                        generator.get_entities_synonyms())
    
chatette(train_samples=400)

### Und nun: neue Tests!

Bringen die umfangreicheren Trainingsdaten wirklich eine Verbesserung? Schauen wir's uns an! Um verschiedene Sprach-Engines zu vergleichen, nutzen wir die folgende Funktion:

In [ ]:
def train_and_test(config=CONFIG_SK, utterances=TEST_UTTERANCES):
    interpreter = train(config, CHATETTE_DIR)
    test(interpreter, utterances)
    return interpreter

interpreter = train_and_test()

Das war die Sprach-Engine mit der Konfiguration `spacy_sklearn`. Die erste und die letzte Test-Äußerung wurden hier falsch verstanden, obwohl sie recht eindeutig klingen. Vergleichen wir das mit dem TensorFlow-Klassifikator:

In [ ]:
CONFIG_TF = """
language: de_core_news_sm
pipeline:
- name: "nlp_spacy"
  case_sensitive: true
- name: "tokenizer_spacy"
- name: "ner_crf"
- name: "ner_synonyms"
- name: "intent_featurizer_count_vectors"
- name: "intent_classifier_tensorflow_embedding"
"""
interpreter = train_and_test(config=CONFIG_TF)

Hier wurde nur die letzte Äußerung nicht verstanden, aber das ist auch nicht weiter verwunderlich.

##  Unser kleiner WetterBot

Experimentieren macht mehr Spaß, wenn es auch mal zischt und knallt. Oder zumindest irgendeine andere Reaktion erfolgt. Und deswegen bauen wir uns einen kleinen WetterBot, der auf die erkannten Intents reagieren kann. Zuerst schreiben wir dafür eine Eingabe-Verarbeitungs-Ausgabe-Schleife. Diese erwartet als Parameter erstens die Sprach-Engine `interpreter` und zweitens ein Python-Dictionary `handlers`, welches jeder Intent-Bezeichnung einen Handler zuordnet. Der Handler wird dann mit dem erkannten Intent aufgerufen und sollte zurückgeben, ob die Schleife fortgeführt werden soll oder nicht:

In [ ]:
def dialog(interpreter, handlers):
    quit = False
    while not quit:
        intent = extract_intent(interpreter.parse(input('>')))
        print('<', intent)
        intent_name = intent[0]
        if intent_name in handlers:
            quit = handlers[intent_name](intent)


Wir implementieren gleich beispielhaft einen Handler für den Intent `Frag_Temperatur`und reagieren auf alle anderen Intents mit einer Standard-Antwort:

In [ ]:
def message(msg, quit=False):
    print(msg)
    return quit

HANDLERS = { 
    'Ende': lambda intent: message('=> Oh, wie schade. Bis bald!', True),
    'Frag_Zeit': lambda intent: message('=> Das ist eine gute Frage.'),
    'Frag_Ort': lambda intent: message('=> Dafür wurde ich nicht programmiert.'),
    'Frag_Temperatur': lambda intent: message('=> Das weiss ich nicht.')
}


Um die Fragen nach den Temperaturen zu beantworten, nutzen wir [Archiv-Daten](ftp://ftp-cdc.dwd.de/pub/CDC/regional_averages_DE/annual/air_temperature_mean/regional_averages_tm_year.txt) des [Deutschen Wetterdienstes](https://www.dwd.de), die wir schon etwas aufbereitet haben. Die Routine `show` gibt die nachgefragten Temperaturdaten je nach Anzahl der angegebenen Jahre und Bundesländer als Liniendiagramm, Balkendiagramm oder in Textform an. Der eigentliche Hander `frag_wert` prüft, ob die angegebenen Jahre und Orte auch zulässig sind und setzt, falls eine der beiden Angaben fehlt, einfach alle Jahre beziehungsweise Bundesländer ein:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import set_matplotlib_formats
%matplotlib inline
set_matplotlib_formats('svg')

sns.set()

DATA_PATH = os.path.join(DATA_DIR, 'temperaturen.txt')
temperature = pd.read_csv(DATA_PATH, index_col=0, sep=';')

def show(times, places):
    if (len(places) == 0) and (len(times) == 0):
        print('Keine zulässigen Orte oder Zeiten')
    elif (len(places) == 1) and (len(times) == 1):
        print(temperature.loc[times, places])
    else:
        if (len(places) > 1) and (len(times) == 1):            
            temperature.loc[times[0], places].plot.barh()
        if (len(places) == 1) and (len(times) > 1):
            temperature.loc[times, places[0]].plot.line()
        if (len(places) > 1) and (len(times) > 1):
            temperature.loc[times, places].plot.line()
            plt.legend(bbox_to_anchor=(1.05,1), loc=2, borderaxespad=0.)
        plt.show()

def frag_temperatur(intent):
    def validate(options, ent_name, fn):
        chosen = [fn(value) for (name, value) in intent[1] if name == ent_name]
        return list(set(options) & set(chosen)) if chosen else options
    places = validate(list(temperature.columns), 'Ort', lambda x:x)
    times = validate(list(temperature.index), 'Zeit', int)
    show(times, places)
    return False

HANDLERS['Frag_Temperatur'] = frag_temperatur

Nun kann der WetterBot getestet werden! Zum Beispiel mit

>  "Wie warm war es in Baden-Württemberg und Sachsen?"

In [ ]:
dialog(interpreter, HANDLERS)

Und jetzt kannt Du loslegen &mdash; der WetterBot kann noch nicht viel, ist aber nun recht einfach zu trainieren! Ein paar Ideen dazu gibt Dir das Notebook mit Aufgaben zu Intent Recognition.

_Viel Spaß und bis bald zu einer neuen Lektion vom codecentric.AI bootcamp!_